### Problem 3 

#### LBG Algorithm for codebook optimization for the given 8x8 Image:
The following steps were followed:

    Initializing the code book
    Training vector generation 
    Calculate distance and find the codebook class
    Calculation Distortion 
    Update the codebook
 
    Threshold value = 0.001 is considered


In [1]:
#Initialize the codebook
codebook=[]
cb = [(60,65,70,75), (60,70,80,90), (65,75,85,95), (60,80,100,120)];
codebook.append(cb)

In [2]:
#Input Image 
import numpy as np
image = np.array([(52,55,61,66,70,61,64,73),(63,59,66,90,109,85,69,72),(62,59,68,113,144,104,66,73),(63,58,71,122,154,106,70,69),
        (67,61,68,104,126,88,68,70),(79,65,60,70,77,68,58,75),(85,71,64,59,55,61,65,83),(87,79,69,68,65,76,78,94)])

In [3]:
#8x8 Image vectorization
def create_training_vectors(image):
    """
    Input  : 8x8 Image array , Reshape the image to 8x8 before inputing
    Output : 16 Training vectors each of size 4x1 (This is obtained after Sliding a 2x2 Mask over the image)
    """
    train_vectors=[]
    row_index = 0
    col_index = 0
    for i in range(1,17):
            train_vectors.append(image[row_index:row_index+2,col_index:col_index+2]) #Sliding the 2x2 Mask
            col_index += 2
            if i%4==0:
                row_index += 2 
                col_index =0 
    #Converting the training arrays into vectors
    tmp = []
    train = []
    count = 0
    for j in range(len(train_vectors)):
        for index in train_vectors[j]:
            for i in index:
                tmp.append(i)
        train.append(tmp)
        tmp=[]
    return train

In [4]:
#Calculate the distance and find the closest class
def cluster_classification(train,cb):
    """
    This function takes two input arguments 
    Input  : Trainvectors obtained from function create_training_vectors by passing the input image array
             Initial Codebook
    This function outputs two output arrays
    Output : Encoded Image array based on the initial codebook 
             Minimum distance array
    """
    summation = 0
    distance = []
    idx = 0
    flag = 0
    key = []
    row_idx = 0
    position = []
    min_dis=[]
    for num in range(0,16): 
        for i in range(0,4):
            for j in range(0,4):
                #print('Train index ', train[num][j])
                #print('Codebook index ',cb[idx][j])
                summation = summation + (train[num][j] - cb[idx][j])**2
                flag +=1
                if flag ==4: 
                   # print('inside if')
                    key.append(idx)
                   # print('distance , ', np.sqrt(summation)) 
                    distance.append(np.sqrt(summation))
                    summation = 0
                    flag = 0
                    idx += 1
        position.append(distance.index(min(distance)))
        min_dis.append(min(distance))
        idx = 0
       # print('position ' , distance.index(min(distance)))
       # print('min distance ',min(distance))
        distance = []
    #print(position)
    #print(min_dis)
    return position,min_dis

In [5]:
def update_codebook(position,train):
    """
    This function takes two input arguments 
    Input  : Position vector obtained from the function cluster_classification
             Trainvectors 
    No return for the function. Updates the codebook by taking the average of the trainvectors with same code mapping.
    """
    #Getting the indicies
    cb_new = []
    indices = []
    temp = []
    for k in range(0,4):
        for i in range(len(position)):
            if position[i] == k:
                temp.append(i)
        indices.append(temp)
        temp=[]
    #print('Indices ',indices)
    #Forming the update array
    tmp_0=[]
    update_array=[]
    for pos in indices:
        for i in pos:
            tmp_0.append(train[i])
        update_array.append(tmp_0)
        tmp_0=[]   
    #Appending to the codebook
    #print('update array ',update_array)
    cb_new=[]
    tmpr = []
    for tupa in range(0,4):
        for i in range(0,4):
                tmpr.append(int(round(np.mean([k[i] for k in update_array[tupa]]))))
        cb_new.append(tmpr)
        tmpr=[]  
    codebook.append(cb_new)

In [6]:
#main code 
#Threshold value is set to 0.001
distortion =[]   #Updates the distortion to update the codebook
encoded_array=[] #Stores the encoded array
train_vectors = create_training_vectors(image) #Create the training Vectors 

#setting an approximate range of 100. This is can be modified for extended usecase 
for i in range(100):
    #print('main function ',i)
    position, min_dis = cluster_classification(train_vectors,codebook[i]) #Grouping the training vectors into clusters
    #Vector Quantized weights
    encoded_img = np.array(position)
    encoded_array.append(np.resize(encoded_img,[4,4]))
    distortion.append(np.mean(min_dis)) #Calculating the average of minimum distance obtained
    if i==0 :
        print('Updating codebook ',i)
        update_codebook(position,train_vectors)
        print(codebook[i])
        print(encoded_array[i])
    else:
        if distortion[i-1] < distortion[i]:  #checks if the previous distortion value is lesser than the current
            print('Distortion D[',i-1,'] ', distortion[i-1])
            print('Distortion D[',i,'] ', distortion[i])
            print('Distortion D[',i-1,'] is less than Distortion D[',i,']')
            print('')
            print('Optimized codebook is ')
            print(codebook[i-1])
            print('Encoded image array: ')
            print(encoded_array[i-1])
            break
        elif ((distortion[i-1]-distortion[i])/distortion[i-1]) < 0.001: #Check for threshold
            print('Distortion D[',i-1,'] ', distortion[i-1])
            print('Distortion D[',i,'] ', distortion[i])
            print('Distortion is less than the threshold value 0.001')
            print('')
            print('Optimized codebook is ')
            print(codebook[i-1])
            print('Encoded image array: ')
            print(encoded_array[i-1])
            break
        else:
            print('Distortion D[',i-1,'] ', distortion[i-1])
            print('Distortion D[',i,'] ', distortion[i])
            print('Updating codebook ',i)  
            update_codebook(position,train_vectors)
            print(codebook[i])
            print(encoded_array[i])

Updating codebook  0
[(60, 65, 70, 75), (60, 70, 80, 90), (65, 75, 85, 95), (60, 80, 100, 120)]
[[0 1 2 0]
 [0 3 3 0]
 [0 0 2 0]
 [2 0 0 2]]
Distortion D[ 0 ]  28.177460077395104
Distortion D[ 1 ]  22.023872976366718
Updating codebook  1
[[63, 68, 66, 68], [61, 66, 66, 90], [86, 76, 88, 82], [106, 108, 112, 114]]
[[0 1 2 0]
 [0 3 3 0]
 [0 0 2 0]
 [2 0 0 1]]
Distortion D[ 1 ]  22.023872976366718
Distortion D[ 2 ]  21.615804226698177
Updating codebook  2
[[63, 68, 66, 68], [63, 74, 72, 92], [94, 73, 91, 77], [106, 108, 112, 114]]
[[0 1 2 0]
 [0 1 3 0]
 [0 0 2 0]
 [2 0 0 1]]
Distortion D[ 2 ]  21.615804226698177
Distortion D[ 3 ]  17.711118378528894
Updating codebook  3
[[63, 68, 66, 68], [65, 87, 72, 102], [94, 73, 91, 77], [144, 104, 154, 106]]
[[0 0 2 0]
 [0 1 3 0]
 [0 0 2 0]
 [2 0 0 1]]
Distortion D[ 3 ]  17.711118378528894
Distortion D[ 4 ]  17.47223623977098
Updating codebook  4
[[63, 68, 66, 70], [66, 98, 74, 108], [94, 73, 91, 77], [144, 104, 154, 106]]
[[0 0 2 0]
 [0 1 3 0]
 [0 0